In [5]:
import pandas as pd
import numpy as np
import joblib
import os
from time import time

from lightgbm import LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [6]:
x_tr_resample = pd.read_csv('../Data/X_train_smote.csv')
y_tr_resample = np.loadtxt("../Data/y_train_smote.csv", delimiter=",")

In [7]:
minmax = MinMaxScaler()
x_minmax_scaled = minmax.fit_transform(x_tr_resample)
joblib.dump(minmax, "../Models/Default/minmax.pkl")

pt = PowerTransformer()
x_pt_scaled = pt.fit_transform(x_tr_resample)
joblib.dump(pt, "../Models/Default/powertransformer.pkl")

['../Models/Default/powertransformer.pkl']

In [8]:
models = {
    "LGBM": LGBMClassifier(n_estimators=100, random_state=42, verbose=-1, force_col_wise=True),
    "ETC": ExtraTreesClassifier(n_estimators=500, random_state=42),
    "SVM": SVC(probability=True, kernel='rbf', C=1, random_state=42),
    "GNB": GaussianNB(var_smoothing=1e-09),
    "CATBOOST": CatBoostClassifier(iterations=1000, verbose=0, random_state=42)
}

In [9]:
model_dir = "../Models/Default"
os.makedirs(model_dir, exist_ok=True)

In [10]:
results = []

In [11]:
for name, model in models.items():
    print(f"Training {name}...")
    start_time = time()
    
    if name == "SVM":
        model.fit(x_minmax_scaled, y_tr_resample)
        y_pred = model.predict(x_minmax_scaled)
        y_pred_proba = model.predict_proba(x_minmax_scaled)[:, 1]
    elif name == "GNB":
        model.fit(x_pt_scaled, y_tr_resample)
        y_pred = model.predict(x_pt_scaled)
        y_pred_proba = model.predict_proba(x_pt_scaled)[:, 1]
    else:
        model.fit(x_tr_resample, y_tr_resample)
        y_pred = model.predict(x_tr_resample)
        y_pred_proba = model.predict_proba(x_tr_resample)[:, 1]
    
    end_time = time()
    
    # Compute Metrics
    accuracy = accuracy_score(y_tr_resample, y_pred) * 100
    precision = precision_score(y_tr_resample, y_pred, zero_division=0) * 100
    recall = recall_score(y_tr_resample, y_pred, zero_division=0) * 100
    f1 = f1_score(y_tr_resample, y_pred, zero_division=0) * 100
    roc_auc = roc_auc_score(y_tr_resample, y_pred_proba) * 100
    computation_time = end_time - start_time
    
    results.append([name, computation_time, accuracy, precision, recall, f1, roc_auc])
    # Save Model
    model_path = os.path.join(model_dir, f"MODEL-{name}.pkl")
    joblib.dump(model, model_path)
    print(f"{name} model saved at {model_path}")

Training LGBM...
LGBM model saved at ../Models/Default\MODEL-LGBM.pkl
Training ETC...
ETC model saved at ../Models/Default\MODEL-ETC.pkl
Training SVM...
SVM model saved at ../Models/Default\MODEL-SVM.pkl
Training GNB...
GNB model saved at ../Models/Default\MODEL-GNB.pkl
Training CATBOOST...
CATBOOST model saved at ../Models/Default\MODEL-CATBOOST.pkl


In [12]:
print("\nFinal Evaluation Results:")
print("=" * 75)
print(f"{'Model':<10} {'Time (s)':<12} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1 Score':<12} {'ROC AUC':<12}")
print("=" * 75)
for res in results:
    print(f"{res[0]:<10} {res[1]:<12.4f} {res[2]:<12.2f} {res[3]:<12.2f} {res[4]:<12.2f} {res[5]:<12.2f} {res[6]:<12.2f}")
print("=" * 75)

print("\nAll models trained, evaluated, and saved successfully!")


Final Evaluation Results:
Model      Time (s)     Accuracy     Precision    Recall       F1 Score     ROC AUC     
LGBM       0.1697       100.00       100.00       100.00       100.00       100.00      
ETC        2.1167       100.00       100.00       100.00       100.00       100.00      
SVM        4.2746       97.40        96.96        97.88        97.41        99.38       
GNB        0.0040       92.45        88.49        97.60        92.82        97.72       
CATBOOST   3.7431       99.94        99.98        99.90        99.94        100.00      

All models trained, evaluated, and saved successfully!
